In [11]:
!pip install --upgrade pip --quiet
!pip install boto3 tqdm pandas --quiet
!pip install geojson

Defaulting to user installation because normal site-packages is not writeable


In [43]:
import boto3
import re
import datetime

import pandas as pd
import random
import json

from collections import defaultdict
from tqdm.notebook import tqdm

s3 = boto3.client('s3')

In [38]:
# paginator = s3.get_paginator('list_objects_v2').paginate(Bucket='doc-trail-camera-footage')
# 
# f = open('keys.txt', 'w')
# for response in tqdm(paginator):
#     for obj in response['Contents']:
#         f.write(obj['Key'] + '\n')

In [6]:
examples = []
for i, key in enumerate(tqdm(open('keys.txt'))):
    key = key.strip()
    if i % 10000 == 0:
        examples += [key]

0it [00:00, ?it/s]

In [4]:
keys = [
    key.strip()
    for key in open('keys.txt')
    if re.match(r'^Processed\/DOC\/Eastern_Forest_Footage.*.JPG$', key.strip())
]

In [5]:
df = defaultdict(list)

for key in tqdm(keys[1:]):
    match = re.match('^.*\/(.*)\/([0-9]*)_(.*).JPG$', key)
    device, timestamp, label = match.group(1, 2, 3)
    
    df['device'].append(device)
    df['timestamp'].append(timestamp)
    df['label'].append(label)
    
df = pd.DataFrame(df)

  0%|          | 0/768431 [00:00<?, ?it/s]

In [6]:
def timestamp_to_month(timestamp):
    date = datetime.datetime.strptime(timestamp, '%Y%m%d%H%M%S')
    return date.strftime('%b %Y')
    
df['month'] = df.timestamp.apply(timestamp_to_month)

In [7]:
df

,device,timestamp,label,month
0,DCamA01_2019_2,20190329141439,human,Mar 2019
1,DCamA01_2019_2,20190329141441,human,Mar 2019
2,DCamA01_2019_2,20190329141444,human,Mar 2019
3,DCamA01_2019_2,20190329141514,human,Mar 2019
4,DCamA01_2019_2,20190329141517,human,Mar 2019
...,...,...,...,...
768426,HCAMG13_2019_3,20190723013714,empty,Jul 2019
768427,HCAMG13_2019_3,20190723013716,empty,Jul 2019
768428,HCAMG13_2019_3,20190723104852,empty,Jul 2019
768429,HCAMG13_2019_3,20190723104855,empty,Jul 2019


In [13]:
records = df.groupby(['device', 'month', 'label']).agg(['count']).reset_index()

In [44]:
report = {
    'devices': defaultdict(dict),
    'detections': defaultdict(lambda: defaultdict(lambda: defaultdict(int))),
}

for device in records.device.unique():
    report['devices'][device]['lat'] = random.uniform(-36.77, -36.79) 
    report['devices'][device]['lon'] = random.uniform(174.83, 174.88)
    
for record in records.itertuples(name=None):
    _, device, month, label, n = record
    if label in ['mouse', 'possum', 'robin', 'tomtit', 'rat', 'cat', 'kea', 'kiwi']:
        label = label.capitalize()
    else:
        continue
        
    report['detections'][month][label][device] = n
    report['detections'][month]['Total'][device] += n
    
json.dump(report, open('report.json', 'w'))